In [1]:
from pyrealm.pmodel import (
    SubdailyScaler,
    memory_effect,
    SubdailyPModel,
    PModelEnvironment,
    PModel,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.pmodel.optimal_chi import OptimalChiPrentice14
from pyrealm.pmodel.functions import calc_ftemp_arrh, calc_ftemp_kphio,calc_modified_arrhenius_factor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.constants import PModelConst
pmodel_const = PModelConst()

pyrealm/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).


In [ ]:
import glob
import pandas as pd
import numpy as np
import os
from pyrealm.pmodel import SubdailyPModel, PModelEnvironment, SubdailyScaler

# ==== replace with your input  i.e. fluxnet2015 + modis LAI ====
fluxnet_root = "FLX_CA-Oas_FLUXNET2015_SUBSET_HH_1996-2010_1-4.csv"
lai_file = "CA-Oas_MCD15A3H_FPAR_LAI_QC.csv"

# read 4-day modis
lai_data = pd.read_csv(lai_file)
lai_data['date'] = pd.to_datetime(lai_data['datetime'], dayfirst=True, errors='coerce')



In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from pyrealm.pmodel import PModelEnvironment, SubdailyPModel, SubdailyScaler

# ==== Helper functions ====
def expand_hourly_to_halfhourly(df):
    """Expand hourly data into half-hourly data"""
    expanded_index = []
    expanded_rows = []
    for ts, row in df.iterrows():
        expanded_index.extend([ts, ts + pd.Timedelta(minutes=30)])
        expanded_rows.extend([row, row])
    expanded = pd.DataFrame(expanded_rows, index=expanded_index)
    expanded = expanded.sort_index()
    return expanded

def get_lai_and_fapar(site_id, flux_times, date_col='date'):
    """
    site_id: string, site name (case-insensitive)
    flux_times: pandas.Series or DatetimeIndex (half-hourly time axis)
    date_col: column name of date in the MODIS table, could be 'date' or 'datetime'
    requires external lai_data: must contain columns ['SiteID', date_col, 'Lai'] (optional 'Fpar')
    """
    # 1) Filter by site (normalize case and whitespace)
    sub = lai_data[lai_data['SiteID'].str.strip().str.lower() == site_id.strip().lower()].copy()
    if sub.empty:
        # No data for this site: return all NaN (or apply default value strategy here)
        n = len(flux_times)
        return np.full(n, np.nan), np.full(n, np.nan)

    # 2) Normalize date & numeric type
    sub[date_col] = pd.to_datetime(sub[date_col], errors='coerce')
    sub = sub.dropna(subset=[date_col])
    sub['Lai'] = pd.to_numeric(sub.get('Lai'), errors='coerce')

    # 3) Aggregate by day, remove duplicates within the same day
    daily_lai = sub.groupby(sub[date_col].dt.normalize())['Lai'].mean()

    # 4) Build complete daily time axis and interpolate (fill edges forward/backward)
    start_day = pd.to_datetime(flux_times.min()).normalize()
    end_day   = pd.to_datetime(flux_times.max()).normalize()
    all_days = pd.date_range(start_day, end_day, freq='D')

    daily_lai = (daily_lai
                 .reindex(all_days)               
                 .interpolate('linear')           
                 .bfill()
                 .ffill())

    # 5) Map to half-hourly time axis (match by day)
    flux_days = pd.to_datetime(pd.Series(flux_times)).dt.normalize()
    lai_series = daily_lai.reindex(flux_days, method='nearest').to_numpy()

    # 6) Calculate fAPAR
    k = 0.5
    fapar = 1 - np.exp(-k * lai_series)

    return lai_series, fapar


# ==== Main loop ====
all_sites_daily = []
all_sites_subdaily = []

file_list = glob.glob(os.path.join(fluxnet_root, "**/*H*.csv"), recursive=True)
print(f"Found {len(file_list)} files")

for filepath in file_list:
    site_id = filepath.split("/")[-1].split("_")[1]
    print(f"Processing {site_id} ...")

    # Read FLUXNET data
    df = pd.read_csv(filepath)
    df['TIMESTAMP_START'] = df['TIMESTAMP_START'].astype(str).str.zfill(12)
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP_START'], format='%Y%m%d%H%M', errors='coerce')
    df = df.set_index('TIMESTAMP')
    df = df.replace(-9999, np.nan).dropna(subset=['TA_F','PA_F','VPD_F','SW_IN_F','GPP_NT_VUT_REF'])

    # Detect resolution and expand
    time_diffs = df.index.to_series().diff().dropna().dt.total_seconds()
    avg_interval = time_diffs.mean()
    if avg_interval > 1800:  # hourly data
        print(f"{site_id}: Detected hourly data, expanding to half-hourly...")
        df = expand_hourly_to_halfhourly(df)
        if site_id == "BR-Sa1":
            print("\n===== BR-Sa1 interpolated Fluxnet DataFrame (first 20 rows) =====")
            print(df.head(20))

    # Regenerate complete half-hourly time index, ensure regularity
    full_time_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='30T')
    df = df.reindex(full_time_index).interpolate().ffill()
    times = df.index.to_numpy()

    # Extract variables
    Tair = np.clip(df['TA_F'].values, -25, None)  # constrain temperature
    patm = df['PA_F'].values * 1000  # kPa -> Pa
    vpd  = df['VPD_F'].values * 100  # kPa -> Pa
    ppfd = df['SW_IN_F'].values * 2.04
    co2  = df['CO2_F_MDS'].fillna(400).values  # default 400 ppm

    # Extract variables and check
    Tair = np.clip(df['TA_F'].values, -25, None)  # constrain temperature range, °C
    patm = df['PA_F'].values * 1000  # kPa → Pa
    vpd  = df['VPD_F'].values * 100  # hPa → Pa
    ppfd = df['SW_IN_F'].values * 2.04  # SW → PPFD
    co2  = df['CO2_F_MDS'].fillna(400).values  # fill missing with 400 ppm

    # Check for NaN or abnormal values
    print(f"Check input variables: Tair[{np.nanmin(Tair)}~{np.nanmax(Tair)}], "
        f"patm[{np.nanmin(patm)}~{np.nanmax(patm)}], "
        f"vpd[{np.nanmin(vpd)}~{np.nanmax(vpd)}], "
        f"ppfd[{np.nanmin(ppfd)}~{np.nanmax(ppfd)}], "
        f"co2[{np.nanmin(co2)}~{np.nanmax(co2)}]")

    # Interpolate LAI & fAPAR
    lai, fapar = get_lai_and_fapar(site_id, pd.Series(df.index))

    # Output first 10 records of LAI and fAPAR
    debug_df = pd.DataFrame({
        'time': df.index[:10],
        'lai': lai[:10],
        'fapar': fapar[:10]
    })
    print(debug_df)

    # SubdailyScaler
    fs_scaler = SubdailyScaler(times)
    fs_scaler.set_window(window_center=np.timedelta64(12, "h"), half_width=np.timedelta64(30, "m"))

    # Model environment & run
    env_subdaily = PModelEnvironment(tc=Tair, patm=patm, vpd=vpd, co2=co2)

    subdailyC3 = SubdailyPModel(env=env_subdaily, method_kphio='temperature',
                                 reference_kphio=0.125, method_optchi='prentice14',
                                 fapar=fapar, ppfd=ppfd, fs_scaler=fs_scaler,
                                 alpha=1/15, allow_holdover=True)

    subdailyC4 = SubdailyPModel(env=env_subdaily, method_kphio='temperature',
                                 reference_kphio=0.125, method_optchi='c4_no_gamma',
                                 fapar=fapar, ppfd=ppfd, fs_scaler=fs_scaler,
                                 alpha=1/15, allow_holdover=True)

    # Subdaily output
    subdaily_df = pd.DataFrame({
        'time': times,
        'GPP_c3': subdailyC3.gpp / 12,
        'GPP_c4': subdailyC4.gpp / 12,
        'vcmax_opt': subdailyC3.subdaily_vcmax25,
        'jmax_opt': subdailyC3.subdaily_jmax25,
        'temperature': Tair,
        'lai': lai,
        'fapar': fapar,
        'FLUXNET_GPP': df['GPP_NT_VUT_REF'].values,
        'PPFD':ppfd
    })
    subdaily_df['site'] = site_id
    subdaily_df['date'] = pd.to_datetime(subdaily_df['time']).dt.date

    # Daily output
    daily_acclimation_temps = fs_scaler.get_daily_means(Tair)
    daily_env  = {
        'time':pd.to_datetime(times).floor("D").unique(),
        'temp_acclim': fs_scaler.get_daily_means(Tair),
        'co2_acclim': fs_scaler.get_daily_means(co2),
        'vpd_acclim': fs_scaler.get_daily_means(vpd),
        'patm_acclim': fs_scaler.get_daily_means(patm),
        'ppfd_acclim': fs_scaler.get_daily_means(ppfd),
        'fapar_acclim': fs_scaler.get_daily_means(fapar),
    }

    # 2. Add this series as new columns into half-hourly DataFrame
    daily_env_df = pd.DataFrame(daily_env)
    daily_env_df['date'] = pd.to_datetime(daily_env['time']).date
    daily_env_df['site'] = site_id
    daily_df = subdaily_df.groupby('date').mean(numeric_only=True).reset_index()
    daily_df['site'] = site_id
    daily_df = pd.merge(daily_env_df,daily_df,on = ['site','date'])
    all_sites_subdaily.append(subdaily_df)
    all_sites_daily.append(daily_df)

# Merge and save
df_subdaily = pd.concat(all_sites_subdaily)
df_daily = pd.concat(all_sites_daily)
#save to local directory
#df_daily.to_csv("all_sites_daily_LAI_FPAR.csv", index=False)
#df_subdaily.to_csv("all_sites_subdaily_LAI_FPAR.csv", index=False)




Found 4 files
Processing MY-PSO ...


pyrealm/2863226631.py:92: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
pyrealm/2863226631.py:93: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.


Check input variables: Tair[19.294~34.77], patm[98925.0~100329.0], vpd[0.0~3362.7000000000003], ppfd[0.0~2365.0332], co2[400.0~400.0]

===== Check LAI and fAPAR: MY-PSO =====
LAI length: 122736, fAPAR length: 122736
LAI range: 0.00 ~ 7.00
fAPAR range: 0.00 ~ 0.97
Missing ratio: LAI=0.00%, fAPAR=0.00%
                 time  lai    fapar
0 2003-01-01 00:00:00  0.8  0.32968
1 2003-01-01 00:30:00  0.8  0.32968
2 2003-01-01 01:00:00  0.8  0.32968
3 2003-01-01 01:30:00  0.8  0.32968
4 2003-01-01 02:00:00  0.8  0.32968
5 2003-01-01 02:30:00  0.8  0.32968
6 2003-01-01 03:00:00  0.8  0.32968
7 2003-01-01 03:30:00  0.8  0.32968
8 2003-01-01 04:00:00  0.8  0.32968
9 2003-01-01 04:30:00  0.8  0.32968


pyrealm/subdaily.py:261: ExperimentalFeatureWarning: 'This is a draft implementation and the API and calculations may change'


Processing BR-Sa1 ...
BR-Sa1: Detected hourly data, expanding to half-hourly...


KeyboardInterrupt: 